In [1]:
from __future__ import print_function
import fwdpy 
recRegions = [fwdpy.Region(1,1e5,0),fwdpy.Region(1e5,2e5,1e-8),fwdpy.Region(2e5,3e5,1e-7)]
for i in recRegions:
    print (i)

beg = 1.000000000, end = 100000.000000000, weight = 0.000000000
beg = 100000.000000000, end = 200000.000000000, weight = 0.001000000
beg = 200000.000000000, end = 300000.000000000, weight = 0.010000000


For this hypothetical example, the region lengths are all identical, and thus an equivalent specification would be this:

In [2]:
recRegions = [fwdpy.Region(1,1e5,0,False),fwdpy.Region(1e5,2e5,1e-8,False),fwdpy.Region(2e5,3e5,1e-7,False)]
for i in recRegions:
    print (i)

beg = 1.000000000, end = 100000.000000000, weight = 0.000000000
beg = 100000.000000000, end = 200000.000000000, weight = 0.000000010
beg = 200000.000000000, end = 300000.000000000, weight = 0.000000100


In [3]:
neutralRegions = [fwdpy.Region(0,1,1)]

The beginning and end positions can be whatever you like:

In [4]:
#With a weight of 1, we're just rescaling the position here.
neutralRegions = [fwdpy.Region(0,100,1)]

To specify variation in the netural mutation process along a sequence, combine multiple regions in your list:


In [5]:
#If coupled=False for the second region, the effect would be that region2's mutation rate per base pair is 10x less than region 1!!
neutralRegions = [fwdpy.Region(beg=0,end=1,weight=1),fwdpy.Region(beg=2,end=12,weight=1,coupled=True)]

Internally, the total "mutational weight" of the first region will be a function of its length, which is 1(1-0)=1.  The second region's total weight will be 1*(12-2)=10, and it will have 10xas many new mutations arising as the first region.

In [6]:
#Let's see what happens if we set coupled=False:
neutralRegions2 = [fwdpy.Region(beg=0,end=1,weight=1),fwdpy.Region(beg=2,end=12,weight=1,coupled=False)]
print("The set with coupled=True:")
for i in neutralRegions:
    print(i)
print("The set with coupled=False:")
for i in neutralRegions2:
    print(i)

The set with coupled=True:
beg = 0.000000000, end = 1.000000000, weight = 1.000000000
beg = 2.000000000, end = 12.000000000, weight = 10.000000000
The set with coupled=False:
beg = 0.000000000, end = 1.000000000, weight = 1.000000000
beg = 2.000000000, end = 12.000000000, weight = 1.000000000


See the difference in the above?  (Look at the "weight" term in the second line of each set.)

In [7]:
#recrate[2] is the hotspot:
recrates = [fwdpy.Region(0.,0.45,1.),fwdpy.Region(0.55,1.,1.,),fwdpy.Region(0.45,0.55,100.)]
for i in recrates:
    print (i)

beg = 0.000000000, end = 0.449999988, weight = 0.449999988
beg = 0.550000012, end = 1.000000000, weight = 0.449999988
beg = 0.449999988, end = 0.550000012, weight = 10.000002384


In [8]:
import fwdpy
import numpy as np
rng = fwdpy.GSLrng(100)
##Some basic parameters
N=1000
theta=100.0
rho=100.0
##All neutral muts are [0,1)
nregions = [ fwdpy.Region(0,1,1) ]
#Selected mutations.  All are additive, to keep this example simple.
##Strongly deleterious mutations to the "left"
##Weaker mutations (2Ns = 10 on average) to the "right"
## 1% of selected mutations will be positively selected
## and uniform throughout the region.  The distribution
## of s will be exponential with mean 1e-3
smodels = [fwdpy.ConstantS(-1,0,0.99/2,-0.1),fwdpy.ExpS(1,2,0.99/2,-10),fwdpy.ExpS(-1,2,0.01,0.001)]
##Recombination models--10x hotspot in the middl
rregions = [fwdpy.Region(-1,1,1),fwdpy.Region(0.45,0.55,10)]
#set up list of population sizes,
#which are NumPy arrays of ints
popsizes = np.array([N],dtype=np.uint32) 
popsizes = np.tile(popsizes,10*N)
pops = fwdpy.evolve_regions(rng,1,N,popsizes[0:],theta/(4*N),0.1*theta/(4*N),rho/(4*N),nregions,smodels,rregions)
#Take a sample of size n = 10 from the population via list comprehension
popSample = [fwdpy.get_samples(rng,i,100) for i in pops]

In [9]:
#Let's convert from tuples to pandas DataFrames.
#Ideally, one would further split each tuple element into a list,
#but this example let's us get the point...
import pandas
neutralMuts = pandas.DataFrame.from_records(popSample[0][0],columns=['pos','genotypes'])
selectedMuts = pandas.DataFrame.from_records(popSample[0][1],columns=['pos','genotypes'])

In [10]:
print(neutralMuts.head())

        pos                                          genotypes
0  0.006588  0000000000000000000000001000010000000000000000...
1  0.012971  1000000000010101000001000010101000000000000010...
2  0.014566  0110000010000010000010010000000111001000010000...
3  0.014729  0000000000000000000000000000000000000000100000...
4  0.020637  0000010000000000000000000000000000000000000000...


In [11]:
print(selectedMuts.head())

        pos                                          genotypes
0  1.016680  0101110011001110000011010011011001010000011100...
1  1.192438  1010001100110001111100101100100110101111100011...
2  1.197182  0101110011001110000011010011011001010000011100...
3  1.365021  0101110011001110000011010011011001010000011100...
4  1.402332  1010001100110001111100101100100110101111100011...
